# 11.1 Object exception

## Exception detail

In [3]:
class General(Exception): pass
class Specific1(General): pass
class Specific2(General): pass

def raiser0():
    X = General() # Raise superclass instance
    raise X
    
def raiser1():
    X = Specific1() # Raise subclass instance
    raise X
    
def raiser2():
    X = Specific2() # Raise different subclass instance
    raise X
    
for func in (raiser0, raiser1, raiser2):
    try:
        func()
    except General: # Match General or any subclass of it
        import sys
        print('caught: %s' % sys.exc_info()[0])

caught: <class '__main__.General'>
caught: <class '__main__.Specific1'>
caught: <class '__main__.Specific2'>


In [69]:
class General(Exception): pass
class Specific1(General): pass
class Specific2(General): pass

def raiser0(): raise General()
def raiser1(): raise Specific1()
def raiser2(): raise Specific2()
    
for func in (raiser0, raiser1, raiser2):
    try:
        func()
    except General as X: # X is the raised instance
        print('caught: %s' % X.__class__)

caught: <class '__main__.General'>
caught: <class '__main__.Specific1'>
caught: <class '__main__.Specific2'>


In [70]:
class Divzero(Exception): pass # Các lớp con đều kế thừa từ Exception
class Oflow(Exception): pass
class Uflow(Exception): pass

## Print and state

In [6]:
# Print agrument
I = IndexError('spam') # Available in object attribute
I.args

('spam',)

In [78]:
print(I)

spam


In [8]:
class E(Exception):pass
try:
    raise E('spam')
except E as X:
    print(X)          # Displays and saves constructor arguments
    print(X.args)
    print(repr(X))

spam
('spam',)
E('spam')


In [83]:
try: # Multiple arguments 
    raise E('spam', 'eggs', 'ham')
except E as X:
    print('%s %s' % (X, X.args))

('spam', 'eggs', 'ham') ('spam', 'eggs', 'ham')


## Custom print display 

In [84]:
class MyBad(Exception): pass
try:
    raise MyBad('Sorry--my mistake!')
except MyBad as X:
    print(X)

Sorry--my mistake!


In [9]:
class MyBad(Exception):
    def __str__(self):
        return 'Always look on the bright side of life...'
try:
    raise MyBad()
except MyBad as X:
    print(X)

Always look on the bright side of life...


## Custom Data and Behavior

In [4]:
class FormatError(Exception):
    def __init__(self, line, file):
        self.line = line
        self.file = file
def parser():
    raise FormatError(42, file='spam.txt')
try:
    parser()
except FormatError as X:
    print('Error at: %s %s' % (X.file, X.line))

Error at: spam.txt 42


In [5]:
class FormatError(Exception): pass # Inherited constructor
def parser():
    raise FormatError(42, 'spam.txt')
try:
    parser()
except FormatError as X:
    print('Error at:', X.args[0], X.args[1])

Error at: 42 spam.txt


## Providing Method

In [11]:
class FormatError(Exception):
    logfile = 'formaterror.txt'
    def __init__(self, line, file):
        self.line = line
        self.file = file
        
    def logerror(self):
        log = open(self.logfile, 'a')
        print('Error at:', self.file, self.line, file=log)
        
def parser():
    raise FormatError(40, 'spam.txt')
    
if __name__ == '__main__':
    try:
        parser()
    except FormatError as exc:
        exc.logerror()

# 11.2 Xử lý với Nested Exception 

## Đối với try/except lồng nhau

<img align='left' src='images/Try_except_nested.jpg'>

In [98]:
def action2():
    print(1 + []) # Generate TypeError
def action1():
    try:
        action2()
    except TypeError: # Most recent matching try
        print('inner try')
try:
    action1()
except TypeError: # Here, only if action1 re-raises
    print('outer try')

inner try


In [99]:
try:
    try:
        action2()
    except TypeError: # Most recent matching try
        print('inner try')
except TypeError: # Here, only if nested handler re-raises
    print('outer try')

inner try


## Đối với try/finally lồng nhau

<img align='left' src='images/Try_finally_nested.jpg'>

```python
try:
    try:
        raise IndexError
    finally:
        print('spam')
finally:
    print('SPAM')
```

Output:

```python
spam
SPAM
----------------------------------------------------------------
IndexError                     Traceback (most recent call last)
<ipython-input-100-5eee2ecdf687> in <module>()
      1 try:
      2     try:
----> 3         raise IndexError
      4     finally:
      5         print('spam')

IndexError:
```

## Lồng nhau với Try/except/finally
```python
def raise1(): raise IndexError
def noraise(): return
def raise2(): raise SyntaxError
for func in (raise1, noraise, raise2):
    print('<%s>' % func.__name__)
    try:
        try:
            func()
        except IndexError:
            print('caught IndexError')
    finally:
        print('finally run')
    print('...')
```
**Output:** 
```python
<raise1>
caught IndexError
finally run
...
<noraise>
finally run
...
<raise2>
finally run
Traceback (most recent call last):

  File "C:\Users\AM\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)

  File "<ipython-input-102-ebc9ca96b541>", line 5, in <module>
    func()

  File "<ipython-input-101-7c18089d6fe8>", line 3, in raise2
    def raise2(): raise SyntaxError

  File "<string>", line unknown
SyntaxError
```

## Exception idoms

In [6]:
class Exitloop(Exception): pass
try:
    while True:
        while True:
            for i in range(10):
                if i > 3: raise Exitloop # break exits just one level
                print('loop3: %s' % i)
            print('loop2')
        print('loop1')
except Exitloop:
    print('continuing')

loop3: 0
loop3: 1
loop3: 2
loop3: 3
continuing


In [106]:
i

4

## Displaying Errors and Tracebacks

In [12]:
import traceback
def inverse(x):
    return 1 / x

try:
    inverse(0)
except Exception:
    traceback.print_exc(file=open('badly.exc', 'w'))
print('Bye')

Bye


Khi đó file badly.exc có nội dung như sau:
```python
Traceback (most recent call last):
  File "<ipython-input-12-c40f2d798988>", line 6, in <module>
    inverse(0)
  File "<ipython-input-12-c40f2d798988>", line 3, in inverse
    return 1 / x
ZeroDivisionError: division by zero
```

## Điểm thoát exit

In [110]:
import sys
def bye():
    sys.exit(40) # Crucial error: abort now!
try:
    bye()
except:
    print('got it') # Oops--we ignored the exit
print('continuing...')

got it
continuing...


# Bài tập Excercise
## A. Quiz
**Câu 1:** Hai điều kiện cần đối với các ngoại lệ do người dùng định nghĩa trong Python là gì?

**Câu 2:** Làm thế nào để các ngoại lệ dựa trên lớp được raise ngang bằng đối với trình xử lý?

**Câu 3:** Kể tên hai cách mà bạn có thể đính kèm thông tin context vào các đối tượng ngoại lệ

**Câu 4:** Hãy kể hai cách mà bạn có thể sửa đổi tin nhắn thông báo lỗi cho các đối tượng ngoại lệ.

**Câu 5:** Câu lệnh assert được tạo ra để làm gì? 
## B. Coding
**Câu 6:** Viết một hàm được gọi là oops gây ra ngoại lệ IndexError khi được gọi. Sau đó viết một hàm doomed() bên có try/except/else để bắt lỗi hàm oops.

**Câu 7:** Thay đổi hàm số oops bạn vừa viết raise ngoại lệ người dụng định nghĩa được gọi là MyError.

---
# <span style= 'color:blue'> Đáp án </span>

**1.** Các ngoại lệ phải được định nghĩa bởi các lớp. Ngoài ra, các lớp ngoại lệ phải được bắt nguồn từ lớp BaseException, kế thừa từ lớp Exception là lớp con của nó.

**2.** Các ngoại lệ dựa trên lớp ngang bằng bởi các mối quan hệ của lớp cha, tên của lớp cha trong exception trình xử lý sẽ bắt các instance của lớp đó.

Do đó, bạn có thể nghĩ về các lớp cha là các danh mục ngoại lệ chung và các lớp con như các loại ngoại lệ cụ thể hơn trong các danh mục đó.

**3.** Bạn có thể đính kèm thông tin ngữ cảnh vào các ngoại lệ dựa trên lớp bằng cách điền vào instance các thuộc tính trong đối tượng instance để raise, thường là tùy chỉnh trong một phương thức khởi tạo lớp. Các lớp ngoại lệ trong built-in cung cấp một phương thức khởi tạo lưu trữ
đối số trên phiên bản tự động (dưới dạng tuple trong thuộc tính args). 

**4.** Tin nhắn thông báo lỗi trong ngoại lệ dựa trên lớp có thể được sửa đổi bằng một tùy chỉnh
\_\_str\_\_ hoặc \_\_repr\_\_ phương thức nạp chồng toán tử. 

**5.** Câu lệnh assert đưa ra một ngoại lệ AssertionError nếu một điều kiện là sai. Nó
hoạt động giống như một câu lệnh raise có điều kiện trong một câu lệnh if.

In [5]:
# 6
def oops():
    raise IndexError()
def doomed():
    try:
        oops()
    except IndexError:
        print("caught a index error")
    else:
        print("no error caught...")
doomed()

caught a index error


In [8]:
# 7
class MyError(Exception):pass

def oops():
    raise MyError('Spam')
    
def doomed():
    try:
        oops()
    except IndexError:
        print("caught a index error")
    except MyError as data:
        print("caught error:",MyError,data)
    else:
        print("no error caught...")
        
doomed()

caught error: <class '__main__.MyError'> Spam
